In [ ]:
#@title Instalação

#%pip uninstall crewai pdfminer.six pdfplumber -y
%pip install crewai pdfminer.six pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.1/327.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 62.7 MB/s eta 

In [ ]:
#@title Configuração inicial
from crewai import Agent, Task, Crew, LLM
from pydantic import BaseModel
import os

api_key = os.gentenv("API_KEY")
model_name = os.getenv("MODEL_NAME")

llm = LLM(model='gemini/gemini-1.5-flash', api_key=api_key)

In [ ]:
#@title Criação de agentes e tasks

analista = Agent(
    name='Analista de artigos cientificos',
    goal="""
        Você é um especialista em análise de artigos científicos.

        Sua tarefa é analisar uma parte especifica de um artigo, somente ela.

        Você deve fornecer dados e/ou uma analise sobre a parte do artigo que foi solicitada.

        Além da parte do artigo, você pode também receber instruções adicionais relacionadas ao artigo, que devem ser seguidas.

        Como resposta, retorne um nome simples para o que foi analisado, e o conteudo da análise. (exemplo: "Introdução" : "A introdução do artigo apresenta...")

        A solicitação de analise é a seguinte (analise o artigo baseado na parte solicitada):
        {solicitacao}

        O artigo que você deve analisar é o seguinte:
        {conteudo_artigo}
    """,
    backstory="""
        Você é um analista de artigos científicos, especializado em analisar partes específicas de artigos.
        Você deve fornecer uma análise detalhada e precisa com base na parte do artigo que foi solicitada.

        Você deve analisar somente o que foi solicitado.
        Se houver instruções adicionais, você deve segui-las.
    """,
    llm=llm,
    role='analista',
)

class AnaliseOutput(BaseModel):
    nome: str
    conteudo: str

analistaTask = Task(
    name='Analise de artigo',
    description='Analisa uma parte específica de um artigo científico.',
    agent=analista,
    output_json=AnaliseOutput,
    expected_output="A análise deve conter um nome simples e o conteúdo da análise. o conteúdo deve ser um texto simples, em markdown se possível",
    inputs=['conteudo_artigo', 'solicitacao']
)

crewAnalise = Crew(
    name='Analise de artigo',
    agents=[analista],
    tasks=[analistaTask],
    llm=llm,
)

In [ ]:
#@title Extração de texto de PDF

from pdfminer.high_level import extract_text
import os

def extrair_texto(caminho_pdf : str):
    return extract_text(caminho_pdf)

def listar_pdfs(pasta="./"):
    return [f for f in os.listdir(pasta) if f.lower().endswith(".pdf")]

# Boas vindas ao analisador de artigos!

Escolha sua opção:

  1. Analise simples (tema central, objetivos, metodologia, resultados, conclusao, resumo, palavras chaves).

  2. Analise customizada (escolha as seções que deseja analisar).

  3. Analise simples + customizada (seções padrão + seções customizadas).

  0. Sair

In [ ]:
#@title Código principal

secoes_para_analisar = []

print("""
Boas vindas ao analisador de artigos.

Escolha sua opção:
      1. Analise simples (tema central, objetivos, metodologia, resultados, conclusao, resumo, palavras chaves).
      2. Analise customizada (escolha as seções que deseja analisar).
      3. Analise simples + customizada (seções padrão + seções customizadas).

      0. Sair
""")

escolha = input("Digite o número da opção desejada: ")

secoes_padrao = [
    "Tema central: Assunto e problema abordado",
    "Objetivos: O que o autor pretende investigar, demonstrar ou dicutir",
    "Metodologia: Quais metodos foram usados e como os dados foram coletados",
    "Resultados: Principais descobertas e conclusões",
    "Conclusões: Quais são as implicações do resultado, o autor responde a pergunta da pesquisa?",
    "Resumo: Resumo geral sobre o artigo",
    "Palavras chave: Principais termos e conceitos abordados no artigo, divida em estrutura de palavras chave",
]

if escolha == '1':
    secoes_para_analisar = secoes_padrao.copy()
elif escolha == '2' or escolha == '3':
    if(escolha == '3'):
        secoes_para_analisar = secoes_padrao.copy()
    print("""
        Para uma analise customizada, é necessário que seja informado o que você deseja analisar.
        Escreva textos diretos sobre o que você deseja analisar, por exemplo:
        - Tema central: Assunto e problema abordado
        - Objetivos: O que o autor pretende investigar, demonstrar ou discutir
          ...

        Você pode também pedir analises mais específicas, como:
            - Quais são os vieses cognitivos abordados no artigo?
            - Quais são os principais argumentos apresentados?
            - Quais áreas de computação são discutidas no artigo?

        Escreva quantas seções você quiser analisar.
        Quando ja tiver terminado, digite 'FIM' para finalizar a entrada.

        Instruções:
            - Cada seção deve ser escrita em uma linha separada.
            - Após escrever uma seção, pressione Enter para adicionar a próxima.
            - Quando terminar, digite 'FIM' e pressione Enter.
    """)
    while True:
        secao = input("Digite a seção que deseja analisar (ou 'FIM' para finalizar): ")
        if secao.strip().upper() == 'FIM':
            break
        secoes_para_analisar.append(secao.strip())


analises = []
for pdf in listar_pdfs():
    print(f"Analisando: {pdf}")
    conteudo_artigo = extrair_texto(pdf)
    analises_artigo = []
    for secao in secoes_para_analisar:
        print(f"Analisando seção: {secao}")
        analise = crewAnalise.kickoff({'conteudo_artigo': conteudo_artigo, 'solicitacao': secao})
        analises_artigo.append(analise)

    print(f"Análise concluída para: {pdf}")
    with open(f'analise_{pdf}.md', 'w') as f:
        for analise in analises_artigo:
            f.write(f"# {analise['nome']}:\n{analise['conteudo']}\n---\n\n")
    print('\n\n')


Boas vindas ao analisador de artigos.

Escolha sua opção:
      1. Analise simples (tema central, objetivos, metodologia, resultados, conclusao, resumo, palavras chaves).
      2. Analise customizada (escolha as seções que deseja analisar).
      3. Analise simples + customizada (seções padrão + seções customizadas).

      0. Sair



In [ ]:




if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file")

if not model_name:
    # Set a default model if not specified
    model_name = 'gemini/gemini-1.5-flash'
    print(f"MODEL_NAME not found in .env file, using default: {model_name}")

print(f"Using model: {model_name}")